Домашнее задание:
Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

("2023-11-20", "Electronics", 100, 12000),
("2023-11-21", "Electronics", 110, 13000),
("2023-11-22", "Electronics", 105, 12500),
("2023-11-20", "Clothing", 300, 15000),
("2023-11-21", "Clothing", 280, 14000),
("2023-11-22", "Clothing", 320, 16000),
("2023-11-20", "Books", 150, 9000),
("2023-11-21", "Books", 200, 12000),
("2023-11-22", "Books", 180, 10000)

Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.
Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [29]:
!pip install pyspark

In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, window
from pyspark.sql import functions as F
from pyspark.sql import Window

In [31]:
#Создание Спарк-сессии
spark = SparkSession.builder.appName("Analysis").getOrCreate()


In [32]:
#Создание дата-фрейм
data = [
    ("2023-11-20", "Electronics", 100, 12000),
("2023-11-21", "Electronics", 110, 13000),
("2023-11-22", "Electronics", 105, 12500),
("2023-11-20", "Clothing", 300, 15000),
("2023-11-21", "Clothing", 280, 14000),
("2023-11-22", "Clothing", 320, 16000),
("2023-11-20", "Books", 150, 9000),
("2023-11-21", "Books", 200, 12000),
("2023-11-22", "Books", 180, 10000)
]

columns = ["date", "category", "quantity", "revenue"]

df = spark.createDataFrame(data).toDF(*columns)

print("DataFrame:")
df.show()

DataFrame:
+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-20|Electronics|     100|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-20|   Clothing|     300|  15000|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|      Books|     150|   9000|
|2023-11-21|      Books|     200|  12000|
|2023-11-22|      Books|     180|  10000|
+----------+-----------+--------+-------+



In [33]:
#Расчёт средней выручки
window_spec = Window.partitionBy("category", "date")
df_avg = df.groupBy("category").agg(avg("revenue").alias("avg_revenue"))
df_avg_cat = df.join(df_avg, on="category")

print("Средняя выручка по категориям товаров:")
df_avg.show()

Средняя выручка по категориям товаров:
+-----------+------------------+
|   category|       avg_revenue|
+-----------+------------------+
|Electronics|           12500.0|
|   Clothing|           15000.0|
|      Books|10333.333333333334|
+-----------+------------------+



In [40]:
#Преобразование через pivot
df_avg_per_day = df.groupBy("category", "date").agg(F.avg("revenue").alias("avg_revenue"))

pivot_df = df_avg_per_day.groupBy("category").pivot("date").agg(F.sum("avg_revenue"))

pivot_df.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics|   12000.0|   13000.0|   12500.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|      Books|    9000.0|   12000.0|   10000.0|
+-----------+----------+----------+----------+



In [41]:
spark.stop()